# Load And Merge Part 1 + Part 2

This notebook loads the latest part-specific CSVs and merges them into one DataFrame.


In [1]:
from pathlib import Path
import pandas as pd

data_dir = Path('.')

part1_candidates = sorted(
    data_dir.glob('emotion_responses_part1*.csv'),
    key=lambda p: p.stat().st_mtime,
)
part2_candidates = sorted(
    data_dir.glob('emotion_responses_part2*.csv'),
    key=lambda p: p.stat().st_mtime,
)

if not part1_candidates:
    raise FileNotFoundError('No emotion_responses_part1*.csv files found.')
if not part2_candidates:
    raise FileNotFoundError('No emotion_responses_part2*.csv files found.')

part1_path = part1_candidates[-1]
part2_path = part2_candidates[-1]

part1_path, part2_path


(PosixPath('emotion_responses_part1.csv'),
 PosixPath('emotion_responses_part2.csv'))

In [2]:
merge_keys = ["participant_id", "session_id", "stimulus_id"]

df_part1 = pd.read_csv(part1_path)
df_part2 = pd.read_csv(part2_path)

df_merged = df_part1.merge(
    df_part2,
    on=merge_keys,
    how="outer",
)

def coalesce(df, col):
    left = f"{col}_x"
    right = f"{col}_y"
    if left in df.columns and right in df.columns:
        df[col] = df[left].combine_first(df[right])
        df.drop(columns=[left, right], inplace=True)

# These are intentionally repeated in both files; keep just one copy.
coalesce(df_merged, "stimulus_type")
coalesce(df_merged, "target_emotion")

analysis_order = [
    "participant_id", "session_id", "stimulus_id", "stimulus_type", "target_emotion",
    "emotion_trial_index", "emotion_rt_ms", "selected_emotion", "accuracy",
    "matching_trial_index", "matching_rt_ms",
    "match_age_rating", "match_masc_rating", "match_attr_rating", "match_quality_rating", "match_artifact_rating",
    "emotion_timestamp", "matching_timestamp",
]

df_merged = df_merged[[c for c in analysis_order if c in df_merged.columns]]
df_merged


,participant_id,session_id,stimulus_id,stimulus_type,target_emotion,emotion_trial_index,emotion_rt_ms,selected_emotion,accuracy,matching_trial_index,matching_rt_ms,match_age_rating,match_masc_rating,match_attr_rating,match_quality_rating,match_artifact_rating,emotion_timestamp,matching_timestamp
0,anon-f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,kid_surprised,ai_kdef_like,surprised,1,6412,happy,incorrect,4,3029,7,6,6,5,5,2026-01-27T15:49:25.470659,2026-01-27T15:49:53.751731
1,anon-f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,woman_happy,real_kdef,happy,2,1279,afraid,incorrect,1,6313,1,3,3,3,3,2026-01-27T15:49:27.734481,2026-01-27T15:49:43.941520
2,anon-f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,man_angry,real_kdef,angry,3,1146,disgusted,incorrect,6,4219,3,3,2,3,2,2026-01-27T15:49:29.848369,2026-01-27T15:50:01.982456
3,anon-f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,woman2_happy,ai_kdef_like,happy,4,3915,happy,correct,2,3194,3,2,2,2,4,2026-01-27T15:49:34.789089,2026-01-27T15:49:47.174309
4,anon-f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,woman_fearful,real_kdef,afraid,5,648,angry,incorrect,3,3474,7,7,7,7,7,2026-01-27T15:49:36.150139,2026-01-27T15:49:50.681104
5,anon-f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,f821e984-aa8a-46f0-b8d0-757c0ea9fdf9,oldman_angry,ai_kdef_like,angry,6,460,afraid,incorrect,5,3944,5,4,3,2,1,2026-01-27T15:49:37.158011,2026-01-27T15:49:57.736323
